In [1]:
# 是mofcom_agricultural products_weekly price的增强版：
# 外加一层循环以爬取左边侧栏不同的周度价格数据集
# 将模块功能定义为函数，以使主循环模块简洁
# 并且最后会按照数据集的中文名称分别导出并命名csv文件，当然也会标记日期

# 提醒1：请在网路条件较好的情况下运行，若爬取过程中出现StaleElementReferenceException异常，增加sleep()等待的秒数或许能解决问题

# 提醒2：在最后主循环中添加判断条件，可只爬取感兴趣的数据集，比如爬取农副产品-周度数据就太耗时了

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep

import pandas as pd 
import numpy as np
import time
import re

In [2]:
# 打开商务部数据中心网站
driver = webdriver.Chrome()
driver.implicitly_wait(10)
url = "https://cif.mofcom.gov.cn/cif/html/dataCenter/index.html?jgnfcprd"
driver.get(url)

In [3]:
# 用一个字典来收集左边侧栏数据集的id与中文名称，
# 本例所要爬取的数据集是：农副产品-周度数据；农产品-...；生产资料-...
# 观察发现，左边侧栏所有数据集的class name都是menu_li，要爬取的数据集的id开头都是li_jg
# 但注意会跟“农副产品-日度数据”的id开头相混淆，所以要剔除
# 日度数据也可以模仿本代码的内侧循环爬取，或者最好参考mofcom_agricultural products_weekly price.ipynp
# 其主要区别在于日历栏的位置和类型栏id
datasets_info = {}
datasets = driver.find_elements(By.CLASS_NAME, 'menu_li')
for ds in datasets:
    ds_id = ds.get_attribute('id')
    # 选择要爬取的数据集，并排除“农副产品-日度数据”
    if ds_id.startswith('li_jg') and ds_id != 'li_jgnfcprd':
        datasets_info.update({ds_id: ds.text})

In [4]:
# 检查用
datasets_info

{'li_jgnfcpzd': '周度数据',
 'li_jgncpgnqh': '国内期货价格',
 'li_jgncpgjxh': '国际现货价格',
 'li_jgncpgjqh': '国际期货价格',
 'li_jgsczlzdjc': '周度监测数据',
 'li_jgsczlgnqh': '国内期货价格',
 'li_jgsczlgjxh': '国际现货价格',
 'li_jgsczlgjqh': '国际期货价格'}

In [5]:
# 设置到最早的开始年份
# 注意，每刷新一个数据集就要重新调整一次日历
def set_start_year():
    # 找到日历栏
    driver.find_element(By.ID, 'nr5_start_date').click()
    # 这里有个iframe，要更换工作目录
    driver.switch_to.frame(driver.find_element(By.XPATH, '/html/body/div[6]/iframe'))
    sleep(1)
    start_year = driver.find_element(By.XPATH, '/html/body/div/div[1]/div[4]/input')
    start_year.clear()
    start_year.send_keys(Keys.ENTER)
    # 完成后退回到主工作目录
    driver.switch_to.default_content()
    sleep(1)

In [6]:
# 收集该数据集所有商品的id、品种（普通黑色字体）、类型（红色字体）。页面上部的类型栏区域
def collect_commodities_info(commodities_info):
    # 这些数据集的类型栏的id恰好一致
    category = driver.find_element(By.ID, 'week_category')
    # 以下之所以这样走依据的是网页结构
    categories = category.find_elements(By.CLASS_NAME, 'yqpzl')
    for cat in categories:
        category_name = cat.find_element(By.CLASS_NAME, 'pzys.yqfont6').text
        commodities = cat.find_element(By.CLASS_NAME, 'ygepz').find_elements(By.TAG_NAME, 'div')
        for com in commodities:
            commodity_name = com.text
            commodity_id = com.get_attribute('id')
            commodities_info.update({commodity_id : [commodity_name, category_name]})
    
    # 检查用
    print(commodities_info)

In [7]:
# 爬取表格内数据。页面下部
def scrape_data(df_list, commodities_info):
    for key, value in commodities_info.items():
        com_id = key
        com_name = value[0]
        com_cat = value[1]
    
        driver.find_element(By.ID, com_id).click()
        sleep(1)

        # 注意：单位随商品而异，尤其是某些数据集（如期货），所以用一个简单的方法来提取单位
        head_unit = driver.find_element(By.XPATH, '//*[@id="nr5_table_head"]/tbody/tr/td[2]')
        unit = head_unit.text[3:-1]

        table = driver.find_element(By.ID, 'nr5_table')
        table_content = table.find_elements(By.TAG_NAME, 'tr')
        temp_list = []
        for tr in table_content:
            temp_list.append(tr.text.split(' '))  
            temp_df = pd.DataFrame(temp_list, columns=['时间','价格','环比增量','环比'])
            temp_df['单位'] = unit
            temp_df['品种'] = com_name
            # 大部分数据集没有一级分类（即只有一个红字按钮且就叫“品种”），就不用多添一列类型了
            if com_cat != '品种':
                temp_df['类型'] = com_cat

        df_list.append(temp_df)

In [8]:
# 将数据输出为csv格式，并自动添加上数据集的中文名
def export_data(df_list, dataset_id, dataset_name):
    # 拼接后列顺序未调整，因为列名比较多样，以后分析数据时用pandas的reindex调整即可
    data = pd.concat(df_list, ignore_index=True)
    # 检查用
    print(data)

    date = time.strftime("%Y-%m-%d", time.localtime()) 
    prefix = ''
    # 观察上文打印出的datasets_info
    # jg = 价格
    if dataset_id.startswith('li_jgnfcp'):
        prefix = '农副产品'
    elif dataset_id.startswith('li_jgncp'):
        prefix = '农产品'
    elif dataset_id.startswith('li_jgsczl'):
        prefix = '生产资料'
        
    data.to_csv('data/'+date+'_'+prefix+'_'+dataset_name, index=False)

In [12]:
# 主循环
for dataset_id, dataset_name in datasets_info.items():
    
    # 此处可以添加对dataset_id的判断，只爬取感兴趣的数据集
    # 尤其当网络不稳定而中断时，增加判断后，继续运行本模块就容易接着爬取
    #if dataset_id != 'li_jgsczlgjqh':
    #    continue
    
    driver.find_element(By.ID, dataset_id).click()
    sleep(1)
    
    set_start_year()
    
    commodities_info = {}
    collect_commodities_info(commodities_info)
    
    df_list = []
    scrape_data(df_list, commodities_info)
    
    export_data(df_list, dataset_id, dataset_name)

{'jgsczlgjqh_224739': ['原油', '品种'], 'jgsczlgjqh_224740': ['燃料油', '品种'], 'jgsczlgjqh_224741': ['轻柴油', '品种'], 'jgsczlgjqh_224743': ['棉花', '品种'], 'jgsczlgjqh_224744': ['天然气', '品种'], 'jgsczlgjqh_224745': ['木材', '品种'], 'jgsczlgjqh_672635': ['天然橡胶(3号)', '品种']}
              时间     价格   环比增量      环比     单位        品种
0     2021-11-26  76.03  -2.95  -3.74%   美元/桶        原油
1     2021-11-19  78.98  -2.97  -3.62%   美元/桶        原油
2     2021-11-12  81.95   0.18   0.22%   美元/桶        原油
3     2021-11-05  81.77  -1.71  -2.05%   美元/桶        原油
4     2021-10-29  83.48   0.47   0.57%   美元/桶        原油
...          ...    ...    ...     ...    ...       ...
5784  2010-06-11  339.6  -24.4   -6.7%  美分/千克  天然橡胶(3号)
5785  2010-06-04    364    -21  -5.45%  美分/千克  天然橡胶(3号)
5786  2010-05-28    385   15.8   4.28%  美分/千克  天然橡胶(3号)
5787  2010-05-21  369.2    9.9   2.76%  美分/千克  天然橡胶(3号)
5788  2010-05-14  359.3   None    None  美分/千克  天然橡胶(3号)

[5789 rows x 6 columns]
